In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cv2

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift

from skimage import transform
from skimage import img_as_float

from skimage.filters import gaussian

from scipy import ndimage

In [2]:
def histogram(image, upper_lim):
    hist = np.zeros((upper_lim + 1,))
    bins = np.array(np.linspace(0, upper_lim, upper_lim + 1), dtype=int)

    for pix in np.array(image.flatten(), dtype=int):
        hist[pix] += 1
        
    return hist, bins

In [3]:
def histogram_cumul(image, upper_lim):
    hist, bins = histogram(image, upper_lim)
    
    hist_cumul = np.zeros((len(hist),))
    
    for i in range(len(hist)):
        hist_cumul[i] = np.sum(hist[0:(i+1)])
        
    return hist_cumul, bins

In [4]:
def histogram_equalization(image, upper_lim):
    cdf, bins = histogram_cumul(image, upper_lim)
    cdf_min = cdf.min()
    MN = image.shape[0]*image.shape[1]
    
    h = np.zeros((len(cdf),))
    
    for v in range(len(cdf)):
        h[v] = np.round(upper_lim*(cdf[v] - cdf_min)/(MN - cdf_min))
    
    h = np.array(h, dtype=int)
    image_new = image.copy()
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image_new[i,j] = h[image_new[i,j]]
    
    return image_new, h, bins

# BGR to GS Conversion, Histogram Equalization, Image Registration

In [198]:
n_image = 7

image_color = []
image_gs = []
image_he = []
image_reg_tf = []

file_name = "./Mobil/"

print("Image loading, GS conversion, histogram equalization: starting...")
for i in range(n_image):
    # load image
    image_color += [cv2.imread(file_name + str(i+1) + ".jpg")]
    # convert to grayscale
    image_gs += [cv2.cvtColor(image_color[-1], cv2.COLOR_BGR2GRAY)]
    # histogram equalization
    img_he, _, _ = histogram_equalization(image_gs[-1], 255)
    image_he += [img_he]
    print(str(100*(i+1)/n_image) + "%")
print("Image loading, GS conversion, histogram equalization: finished.")

print("Image registration: starting...")
# image registration
for i in range(n_image-1):
    shift, _, _ = phase_cross_correlation(image_he[0], image_he[i+1])
    tform = transform.EuclideanTransform(
        #rotation=np.pi / 12.,
        translation = (np.round(shift)[1], np.round(shift)[0])
        )
    img_reg = img_as_float(image_color[i+1])
    img_tf = transform.warp(img_reg, tform.inverse)
    image_reg_tf += [img_tf]
    cv2.imwrite(file_name + str(i+2) + "_reg.jpg", img_tf*255)
    print(str(100*(i+1)/(n_image-1)) + "%")
print("Image registration: finished.")

Image loading, GS conversion, histogram equalization: starting...
14.285714285714286%
28.571428571428573%
42.857142857142854%
57.142857142857146%
71.42857142857143%
85.71428571428571%
100.0%
Image loading, GS conversion, histogram equalization: finished.
Image registration: starting...
16.666666666666668%
33.333333333333336%
50.0%
66.66666666666667%
83.33333333333333%
100.0%
Image registration: finished.


# HDR

## Mobil/Car

In [21]:
# Read all the files with OpenCV
file_name = "./Mobil/"
files = ['2_reg.jpg', '4_reg.jpg', '6_reg.jpg']
for i in range(len(files)):
    files[i] = file_name + files[i]
    
images = list([cv2.imread(f) for f in files])

# Compute the exposure times in seconds
exposures = np.float32([1. / t for t in [10, 40, 160]])

In [22]:
# Exposure fusion using Mertens
merge_mertens = cv2.createMergeMertens()
merge_mertens.setContrastWeight(0.8)
merge_mertens.setSaturationWeight(0.8)
merge_mertens.setExposureWeight(0.5)

res_mertens = merge_mertens.process(images)

In [23]:
# Convert datatype to 8-bit
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')

In [24]:
# Apply gaussian filter
filtered_mertens = 255*gaussian(res_mertens_8bit.copy(), sigma=1, multichannel=True)

In [25]:
# Save the images

cv2.imwrite(file_name + "fusion_mertens.jpg", res_mertens_8bit)

cv2.imwrite(file_name + "fusion_mertens_filtered.jpg", filtered_mertens)

True

## Kebun/Garden

In [5]:
# Read all the files with OpenCV
file_name = "./Kebun/"
files = ['4_reg.jpg', '6_reg.jpg', '8_reg.jpg']
for i in range(len(files)):
    files[i] = file_name + files[i]
    
images = list([cv2.imread(f) for f in files])

# Compute the exposure times in seconds
exposures = np.float32([2, 1/2, 1/8])

In [17]:
# Exposure fusion using Mertens
merge_mertens = cv2.createMergeMertens()
merge_mertens.setContrastWeight(0.8)
merge_mertens.setSaturationWeight(0.8)
merge_mertens.setExposureWeight(0.5)

res_mertens = merge_mertens.process(images)

In [18]:
# Convert datatype to 8-bit
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')

In [19]:
# Apply gaussian filter
filtered_mertens = 255*gaussian(res_mertens_8bit.copy(), sigma=1, multichannel=True)

In [20]:
# Save the images

cv2.imwrite(file_name + "fusion_mertens.jpg", res_mertens_8bit)

cv2.imwrite(file_name + "fusion_mertens_filtered.jpg", filtered_mertens)

True